#Problem

The data scientists at BigMart have collected 2013 sales data for 1559 products across 10 stores in different cities. Also, certain attributes of each product and store have been defined. The aim is to build a predictive model and find out the sales of each product at a particular store.

Using this model, BigMart will try to understand the properties of products and stores which play a key role in increasing sales.

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline

#Read files
test = pd.read_csv('Test.csv')
train = pd.read_csv('Train.csv')

In [3]:
#combine the csv for data cleansing & exploration 
train['source'] = 'train'
test['source'] = 'test'
data = pd.concat([test, train], ignore_index = True, sort = False)
test.shape, train.shape, data.shape

((5681, 12), (8523, 13), (14204, 13))

In [247]:
#null values for each variables 
#ignore the missing values in Item_Outlet_Sales since its the target value that needs to be tested
data.apply(lambda x : sum(x.isnull()))

Item_Identifier                 0
Item_Weight                  2439
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
source                          0
Item_Outlet_Sales            5681
dtype: int64

In [248]:
#descriptive statistics for numerical variables
#Item_Visibility - min is 0 which is not possible since if an item is sold in an outlet it has to be present there
#Item_Weight - missing values 2439
#Outlet_Establishment_Year - ranges from 1985 to 2009. Has to be changed to categorical as old, new so its easier
data.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,11765.000000,14204.000000,14204.000000,14204.000000,8523.000000
mean,12.792854,0.065953,141.004977,1997.830681,2181.288914
std,4.652502,0.051459,62.086938,8.371664,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.710000,0.027036,94.012000,1987.000000,834.247400
50%,12.600000,0.054021,142.247000,1999.000000,1794.331000
75%,16.750000,0.094037,185.855600,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [249]:
#unique values for categorical variables
#1559 product sales across 10 cities as in the problem
data.nunique()

Item_Identifier               1559
Item_Weight                    415
Item_Fat_Content                 5
Item_Visibility              13006
Item_Type                       16
Item_MRP                      8052
Outlet_Identifier               10
Outlet_Establishment_Year        9
Outlet_Size                      3
Outlet_Location_Type             3
Outlet_Type                      4
source                           2
Item_Outlet_Sales             3493
dtype: int64

In [250]:
#item_fat_content - has different representation for same category (Low Fat, lf, low fat)
#item_type - certain categories can be combined since their difference in count is insignificant
#outlet_type - supermarket tier 2 &3 can be combined as one. it has to be checked if its a good idea
#check the frequency of the categorical variables
categorical_variables = [x for x in data.dtypes.index if data.dtypes[x] == 'object']

#remove source, item_identifier, outlet_identifier - since we know their values
categorical_variables = [x for x in categorical_variables 
                         if x not in ['Item_Identifier', 'Outlet_Identifier', 'source']]

#print the frequencies for the categorical_variables
for col in categorical_variables:
    print('Frequency of %s'%col)
    print(data[col].value_counts())
    print('\n')

Frequency of Item_Fat_Content
Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64


Frequency of Item_Type
Fruits and Vegetables    2013
Snack Foods              1989
Household                1548
Frozen Foods             1426
Dairy                    1136
Baking Goods             1086
Canned                   1084
Health and Hygiene        858
Meat                      736
Soft Drinks               726
Breads                    416
Hard Drinks               362
Others                    280
Starchy Foods             269
Breakfast                 186
Seafood                    89
Name: Item_Type, dtype: int64


Frequency of Outlet_Size
Medium    4655
Small     3980
High      1553
Name: Outlet_Size, dtype: int64


Frequency of Outlet_Location_Type
Tier 3    5583
Tier 2    4641
Tier 1    3980
Name: Outlet_Location_Type, dtype: int64


Frequency of Outlet_Type
Supermarket Type1    9294
Grocery Store        1805
Supermarket T

In [251]:
#Average weight by item - default aggfunc for pivot_table is mean
item_weight_avg = data.pivot_table(values = 'Item_Weight', index = 'Item_Identifier')

#store the missing item_weights
miss_bool = data['Item_Weight'].isnull()

#Impute missing weight with mean weight by item and check for the # of missing values
print('original missing: %d'%sum(miss_bool))
data.loc[miss_bool, 'Item_Weight'] = data.loc[miss_bool, 'Item_Identifier'].apply(lambda x : item_weight_avg.loc[x])
print('missing after imputation: %d' %sum(data['Item_Weight'].isnull()))

original missing: 2439
missing after imputation: 0


In [252]:
#Import mode function:
from scipy.stats import mode

#Determing the mode for each
outlet_size_mode = data.pivot_table(values='Outlet_Size', columns='Outlet_Type',aggfunc=(lambda x: x.mode()))
print('Mode for each Outlet_Type')
print(outlet_size_mode)

#Get boolean variable for missing outlet_size
miss_bool = data['Outlet_Size'].isnull()

#impute the missing value and check the count of missing values
print('Missing outlet_size before imputation %d' %sum(miss_bool))
data.loc[miss_bool, 'Outlet_Size'] = data.loc[miss_bool, 'Outlet_Type'].apply(lambda x : outlet_size_mode[x])
print('Missing outlet_size after imputation %d' %sum(data['Outlet_Size'].isnull()))

Mode for each Outlet_Type
Outlet_Type Grocery Store Supermarket Type1 Supermarket Type2  \
Outlet_Size         Small             Small            Medium   

Outlet_Type Supermarket Type3  
Outlet_Size            Medium  
Missing outlet_size before imputation 4016
Missing outlet_size after imputation 0


In [253]:
#Evaluating the idea of combining supermarket type2 and type3
#since the means of type2 and type3 are significantly different lets leave it as it is
data.pivot_table(values = 'Item_Outlet_Sales', index = 'Outlet_Type')

,Item_Outlet_Sales
Outlet_Type,
Grocery Store,339.828500
Supermarket Type1,2316.181148
Supermarket Type2,1995.498739
Supermarket Type3,3694.038558


In [254]:
#imputing values for Item_Visibility with 0.0
#find the mean of item_visibility by item
visibility_mean = data.pivot_table(values = 'Item_Visibility', index = 'Item_Identifier')

#bool for missing item_visibility
miss_bool = data['Item_Visibility'] == 0.0

#Impute missing Item_visibility and check for the number of missing values before and after
print('Missing Item_Visibility before imputation %d' %sum(miss_bool))
data.loc[miss_bool, 'Item_Visibility'] = data.loc[miss_bool, 'Item_Identifier'].apply(lambda x:visibility_mean.loc[x])
print('Missing Item_Visibility after imputation %d' %sum(data['Item_Visibility'] == 0.0))

Missing Item_Visibility before imputation 879
Missing Item_Visibility after imputation 0


In [255]:
#Determine another variable with means ratio - this gives a comparison of the visibility of a product 
#at one store with that of its mean across all stores
data['Item_Visibility_MeanRatio'] = data.apply(lambda x: x['Item_Visibility']/
                                               visibility_mean.loc[x['Item_Identifier']], axis=1)
data['Item_Visibility_MeanRatio'].describe()

count    14204.000000
mean         1.061884
std          0.235907
min          0.844563
25%          0.925131
50%          0.999070
75%          1.042007
max          3.010094
Name: Item_Visibility_MeanRatio, dtype: float64

In [256]:
#Get the first two character from Item_Identifier
data['Item_Type_Combined'] = data['Item_Identifier'].apply(lambda x : x[:2])
#Rename them to more intuitive categories:
data['Item_Type_Combined'] = data['Item_Type_Combined'].map({'FD':'Food',
                                                             'NC':'Non-Consumable',
                                                             'DR':'Drinks'})
data['Item_Type_Combined'].value_counts()

Food              10201
Non-Consumable     2686
Drinks             1317
Name: Item_Type_Combined, dtype: int64

In [257]:
#Determine how old the stores were in 2013 (thats the time of sales data)
#Stores operated are 4-28 years old
data['Outlet_Years'] = 2013 - data['Outlet_Establishment_Year']
data['Outlet_Years'].describe()

count    14204.000000
mean        15.169319
std          8.371664
min          4.000000
25%          9.000000
50%         14.000000
75%         26.000000
max         28.000000
Name: Outlet_Years, dtype: float64

In [258]:
#Change categories of low fat:
print('Original Categories:')
print(data['Item_Fat_Content'].value_counts())

print('\nModified Categories:')
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                             'reg':'Regular',
                                                             'low fat':'Low Fat'})
print(data['Item_Fat_Content'].value_counts())

Original Categories:
Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

Modified Categories:
Low Fat    9185
Regular    5019
Name: Item_Fat_Content, dtype: int64


In [259]:
#non-consumable has low fat as Item_Fat_Content values which should be changed
data.loc[data['Item_Type_Combined'] == 'Non-Consumable', 'Item_Fat_Content'] = 'Non-edible'
data['Item_Fat_Content'].value_counts()

Low Fat       6499
Regular       5019
Non-edible    2686
Name: Item_Fat_Content, dtype: int64

In [260]:
#label encoder and one hot coding of data
#import library
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

#New outlet variable 
data['outlet'] = le.fit_transform(data['Outlet_Identifier'])
var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','outlet']
le = LabelEncoder()
for i in var_mod:
    data[i] = le.fit_transform(data[i])

data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,source,Item_Outlet_Sales,Item_Visibility_MeanRatio,Item_Type_Combined,Outlet_Years,outlet
0,FDW58,20.750,0,0.007565,Snack Foods,107.8622,OUT049,1999,1,0,1,test,NaN,1.029192,1,14,9
1,FDW14,8.300,2,0.038428,Dairy,87.3198,OUT017,2007,2,1,1,test,NaN,1.130311,1,6,2
2,NCN55,14.600,1,0.099575,Others,241.7538,OUT010,1998,2,2,0,test,NaN,1.735215,2,15,0
3,FDQ58,7.315,0,0.015388,Snack Foods,155.0340,OUT017,2007,2,1,1,test,NaN,1.291577,1,6,2
4,FDY38,13.600,2,0.118599,Dairy,234.2300,OUT027,1985,1,2,3,test,NaN,0.917824,1,28,5


In [261]:
#one hot coding 
#creates dummy variables one for each category. 
data = pd.get_dummies(data, columns=['Item_Fat_Content','Outlet_Location_Type',
                                      'Outlet_Size','Item_Type_Combined','Outlet_Type','outlet'])
data.head()

,Item_Identifier,Item_Weight,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,source,Item_Outlet_Sales,Item_Visibility_MeanRatio,...,outlet_0,outlet_1,outlet_2,outlet_3,outlet_4,outlet_5,outlet_6,outlet_7,outlet_8,outlet_9
0,FDW58,20.750,0.007565,Snack Foods,107.8622,OUT049,1999,test,NaN,1.029192,...,0,0,0,0,0,0,0,0,0,1
1,FDW14,8.300,0.038428,Dairy,87.3198,OUT017,2007,test,NaN,1.130311,...,0,0,1,0,0,0,0,0,0,0
2,NCN55,14.600,0.099575,Others,241.7538,OUT010,1998,test,NaN,1.735215,...,1,0,0,0,0,0,0,0,0,0
3,FDQ58,7.315,0.015388,Snack Foods,155.0340,OUT017,2007,test,NaN,1.291577,...,0,0,1,0,0,0,0,0,0,0
4,FDY38,13.600,0.118599,Dairy,234.2300,OUT027,1985,test,NaN,0.917824,...,0,0,0,0,0,1,0,0,0,0


In [262]:
#object(category) datatype is converted into float/uint
data.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
source                        object
Item_Outlet_Sales            float64
Item_Visibility_MeanRatio    float64
Outlet_Years                   int64
Item_Fat_Content_0             uint8
Item_Fat_Content_1             uint8
Item_Fat_Content_2             uint8
Outlet_Location_Type_0         uint8
Outlet_Location_Type_1         uint8
Outlet_Location_Type_2         uint8
Outlet_Size_0                  uint8
Outlet_Size_1                  uint8
Outlet_Size_2                  uint8
Item_Type_Combined_0           uint8
Item_Type_Combined_1           uint8
Item_Type_Combined_2           uint8
Outlet_Type_0                  uint8
Outlet_Type_1                  uint8
Outlet_Type_2                  uint8
Outlet_Type_3                  uint8
o

In [263]:
#new columns created for each category
data.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Visibility', 'Item_Type',
       'Item_MRP', 'Outlet_Identifier', 'Outlet_Establishment_Year', 'source',
       'Item_Outlet_Sales', 'Item_Visibility_MeanRatio', 'Outlet_Years',
       'Item_Fat_Content_0', 'Item_Fat_Content_1', 'Item_Fat_Content_2',
       'Outlet_Location_Type_0', 'Outlet_Location_Type_1',
       'Outlet_Location_Type_2', 'Outlet_Size_0', 'Outlet_Size_1',
       'Outlet_Size_2', 'Item_Type_Combined_0', 'Item_Type_Combined_1',
       'Item_Type_Combined_2', 'Outlet_Type_0', 'Outlet_Type_1',
       'Outlet_Type_2', 'Outlet_Type_3', 'outlet_0', 'outlet_1', 'outlet_2',
       'outlet_3', 'outlet_4', 'outlet_5', 'outlet_6', 'outlet_7', 'outlet_8',
       'outlet_9'],
      dtype='object')

In [264]:
#3 columns are created for each category; each category has 1 for that category values and 0 for the rest
data[['Item_Fat_Content_0', 'Item_Fat_Content_1','Item_Fat_Content_2']].head(10)

,Item_Fat_Content_0,Item_Fat_Content_1,Item_Fat_Content_2
0,1,0,0
1,0,0,1
2,0,1,0
3,1,0,0
4,0,0,1
5,0,0,1
6,0,0,1
7,1,0,0
8,0,0,1
9,1,0,0


In [265]:
#drop the columns that has been converted into different types
data.drop(['Item_Type','Outlet_Establishment_Year'],axis=1,inplace=True)

In [274]:
#divide into train and test
train = data.loc[data['source'] == 'train']
test = data.loc[data['source']== 'test']

#Drop unnecessary columns:
test.drop(['Item_Outlet_Sales','source'],axis=1,inplace=True)
train.drop(['source'],axis=1,inplace=True)

/Users/kriti/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [275]:
#Export files as modified versions:
train.to_csv("train_modified.csv",index=False)
test.to_csv("test_modified.csv",index=False)

In [277]:
#model building 
#basic modeling
#mean sales
mean_sales = train['Item_Outlet_Sales'].mean()
mean_sales

2181.2889135750365